In [7]:
import torch
import numpy as np

## 3.1 Автоматическое дифференцирование в `torch`

3.1.1 Воспользовавшись классами `Neuron` и `SquaredLoss` из задачи 2.4.1 и автоматическим дифференцированием, которое предоставляет `torch`, решить задачу регрессии. Для оптимизации использовать стохастический градиетный спуск.

In [22]:
class Neuron:
    def __init__(self, n_inputs):
        #Атрибут weights
        self.W = torch.randn(n_inputs)
        #Атрибут bias
        self.B = torch.randn(1)

    def forward(self, inputs):
        self.inputs = inputs
        return torch.matmul(inputs, self.W.T) + self.B

    def backward(self, dvalue):

        self.dweights = dvalue * self.inputs
        self.dinput =  dvalue * self.W
        self.dbias = dvalue

        # Возвращаем градиент весов и смещения
        return self.dweights, self.dbias

In [23]:
class SquaredLoss:
    def forward(self, y_pred, y_true):
        self.y_pred = torch.tensor(y_pred, requires_grad=True)
        y_true = torch.tensor(y_true)
        z = (self.y_pred - y_true) ** 2
        self.z = z
        return z

    def backward(self):
        self.z.backward()
        self.dinput = self.y_pred.grad

In [24]:
from sklearn.datasets import make_regression

X, y, coef = make_regression(n_features=4, n_informative=4, coef=True, bias=0.5)
X = torch.from_numpy(X).to(dtype=torch.float32)
y = torch.from_numpy(y).to(dtype=torch.float32)

In [25]:
# <размерность элемента выборки >
n_inputs = 4
# скорость обучения
learning_rate = 0.01
# количество эпох
n_epoch = 50
#размер пакета
batch_size = 10

neuron = Neuron(n_inputs)
loss = SquaredLoss()

losses = []
for epoch in range(n_epoch):
    sample = torch.randint(0, X.shape[0], size=(batch_size,))
    for x_example, y_example in zip(X[sample], y[sample]):
        # Активация
        y_pred = neuron.forward(x_example)
        curr_loss = loss.forward(y_pred, y_example)
        losses.append(curr_loss)

        # Обратное распространение
        loss.backward()
        dweights, dbias = neuron.backward(loss.dinput)

        # Обновление вестов
        neuron.W -= learning_rate * dweights
        neuron.B -= learning_rate * dbias
        print(f"Epoch {epoch} loss -> {curr_loss[0]}")

C:\Users\Danila\AppData\Local\Temp\ipykernel_12816\1712833060.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y_pred = torch.tensor(y_pred, requires_grad=True)
C:\Users\Danila\AppData\Local\Temp\ipykernel_12816\1712833060.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_true = torch.tensor(y_true)


Epoch 0 loss -> 42.639652252197266
Epoch 0 loss -> 812.2446899414062
Epoch 0 loss -> 3148.610107421875
Epoch 0 loss -> 2100.96435546875
Epoch 0 loss -> 485.009521484375
Epoch 0 loss -> 93.40496063232422
Epoch 0 loss -> 7461.3837890625
Epoch 0 loss -> 27.271696090698242
Epoch 0 loss -> 1773.739501953125
Epoch 0 loss -> 2919.37646484375
Epoch 1 loss -> 1483.7337646484375
Epoch 1 loss -> 1863.12109375
Epoch 1 loss -> 14.973762512207031
Epoch 1 loss -> 6003.92041015625
Epoch 1 loss -> 124.45703887939453
Epoch 1 loss -> 0.6290546655654907
Epoch 1 loss -> 11433.376953125
Epoch 1 loss -> 4333.65234375
Epoch 1 loss -> 954.1820678710938
Epoch 1 loss -> 1589.8525390625
Epoch 2 loss -> 4326.564453125
Epoch 2 loss -> 469.353759765625
Epoch 2 loss -> 6271.67529296875
Epoch 2 loss -> 1423.5123291015625
Epoch 2 loss -> 7396.15771484375
Epoch 2 loss -> 885.2360229492188
Epoch 2 loss -> 77.0230484008789
Epoch 2 loss -> 283.11602783203125
Epoch 2 loss -> 953.9406127929688
Epoch 2 loss -> 12448.765625
Ep

 3.1.2 Воспользовавшись классами `Linear` и `MSELoss` из задачи 2.1.4 и 2.3.1, `ReLU` из 2.2.1 и автоматическим дифференцированием, которое предоставляет `torch`, решить задачу регрессии. Для оптимизации использовать пакетный градиентный спуск. Вывести график функции потерь в зависимости от номера эпохи. Вывести на одном графике исходные данные и предсказанные значения.

In [ ]:
X = torch.linspace(0, 1, 100).view(-1, 1)
y = torch.sin(2 * np.pi * X) + 0.1 * torch.rand(X.size()) 

## 3.2 Алгоритмы оптимизации в `torch.optim`

In [28]:
import torch.optim as optim

3.2.1 Решить задачу 3.1.1, воспользовавшись оптимизатором `optim.SDG` для применения стохастического градиентого спуска

3.2.2 Решить задачу 3.1.2, воспользовавшись оптимизатором `optim.Adam` для применения пакетного градиентого спуска. Вывести график функции потерь в зависимости от номера эпохи. Вывести на одном графике исходные данные и предсказанные значения.

## 3.3 Построение сетей при помощи `torch.nn`

In [ ]:
import torch.nn as nn

3.3.1 Решить задачу регрессии, соблюдая следующие условия:

1. Оформить нейронную сеть в виде класса - наследника `nn.Module`
2. При создании сети использовать готовые блоки из `torch.nn`: слои, функции активации, функции потерь и т.д.
3. Для оптимизации использовать любой алгоритм оптимизации из `torch.optim` 

In [26]:
X = torch.linspace(0, 1, 100).view(-1, 1)
y = torch.sin(2 * np.pi * X) + 0.1 * torch.rand(X.size())

In [29]:
class SineNet(torch.nn.Module):
    def __init__(self, n_hidden_neurons):
        super(SineNet, self).__init__()
        self.fc1 = torch.nn.Linear(1, n_hidden_neurons)
        self.act1 = torch.nn.Sigmoid()
        self.fc2 = torch.nn.Linear(n_hidden_neurons, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act1(x)
        x = self.fc2(x)
        return x

neuron = SineNet(3)
optimizer = optim.Adam(neuron.parameters(), lr=0.01)
loss = torch.nn.MSELoss()

for epoch in range(2000):
    optimizer.zero_grad()

    y_pred = neuron.forward(X)
    loss_val = loss(y_pred, y)

    loss_val.backward()

    optimizer.step()

    if epoch % 100 == 0: print(f"Epoch {epoch} loss: {loss_val}")

Epoch 0 loss: 0.554947018623352
Epoch 100 loss: 0.3385131359100342
Epoch 200 loss: 0.18949802219867706
Epoch 300 loss: 0.18628758192062378
Epoch 400 loss: 0.18250218033790588
Epoch 500 loss: 0.17690861225128174
Epoch 600 loss: 0.1686611771583557
Epoch 700 loss: 0.15727734565734863
Epoch 800 loss: 0.14443351328372955
Epoch 900 loss: 0.1323504000902176
Epoch 1000 loss: 0.12031371891498566
Epoch 1100 loss: 0.10165800899267197
Epoch 1200 loss: 0.07109122723340988
Epoch 1300 loss: 0.043266259133815765
Epoch 1400 loss: 0.025755923241376877
Epoch 1500 loss: 0.017111917957663536
Epoch 1600 loss: 0.0133848050609231
Epoch 1700 loss: 0.01180289313197136
Epoch 1800 loss: 0.010959554463624954
Epoch 1900 loss: 0.010262860916554928


3.3.2 Решить задачу регрессии, соблюдая следующие условия:

1. Оформить нейронную сеть в виде объекта `nn.Sequential`
2. При создании сети использовать готовые блоки из `torch.nn`: слои, функции активации, функции потерь и т.д.
3. Для оптимизации использовать любой алгоритм оптимизации из `torch.optim` 

In [ ]:
X = torch.linspace(0, 1, 100).view(-1, 1)
y = torch.sin(2 * np.pi * X) + 0.1 * torch.rand(X.size())

layers = [
    torch.nn.Linear(1, 5),
    torch.nn.Sigmoid(),
    torch.nn.Linear(5, 1)
]
model = torch.nn.Sequential(*layers)

optimizer = optim.Adam(model.parameters(), lr=0.01)
loss = torch.nn.MSELoss()

for epoch in range(2000):
    optimizer.zero_grad()

    y_pred = model.forward(X)
    loss_val = loss(y_pred, y)

    loss_val.backward()

    optimizer.step()

    if epoch % 100 == 0: print(f"Epoch {epoch} loss: {loss_val}")

## 3.4. Datasets and dataloaders

In [31]:
from torch.utils.data import Dataset, DataLoader

    3.4.1 Создать датасет, поставляющий данные из задачи 3.1.2.

    Создать `DataLoader` на основе этого датасета и проверить работоспособность.

    Воспользовавшись результатами 3.3.1 (или 3.3.2) обучите модель, пользуясь мини-пакетным градиентным спуском с размером пакета (`batch_size`) = 10

In [33]:
class SinDataset(Dataset):
    def __init__(self):
        self.X = torch.linspace(0, 1, 100).view(-1, 1)
        self.y = torch.sin(2 * np.pi * self.X) + 0.1 * torch.rand(self.X.size())

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [37]:
sin = SinDataset()
X, y = sin.__getitem__(idx=10)

In [38]:
class SineNet(torch.nn.Module):
    def __init__(self, n_hidden_neurons):
        super(SineNet, self).__init__()
        self.fc1 = torch.nn.Linear(1, n_hidden_neurons)
        self.act1 = torch.nn.Sigmoid()
        self.fc2 = torch.nn.Linear(n_hidden_neurons, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act1(x)
        x = self.fc2(x)
        return x

neuron = SineNet(3)
optimizer = optim.Adam(neuron.parameters(), lr=0.01)
loss = torch.nn.MSELoss()

for epoch in range(2000):
    optimizer.zero_grad()

    y_pred = neuron.forward(X)
    loss_val = loss(y_pred, y)

    loss_val.backward()

    optimizer.step()

    if epoch % 100 == 0: print(f"Epoch {epoch} loss: {loss_val}")

Epoch 0 loss: 0.6816445589065552
Epoch 100 loss: 6.212790594872786e-06
Epoch 200 loss: 7.29048821312972e-10
Epoch 300 loss: 1.4210854715202004e-14
Epoch 400 loss: 0.0
Epoch 500 loss: 0.0
Epoch 600 loss: 0.0
Epoch 700 loss: 0.0
Epoch 800 loss: 0.0
Epoch 900 loss: 0.0
Epoch 1000 loss: 0.0
Epoch 1100 loss: 0.0
Epoch 1200 loss: 0.0
Epoch 1300 loss: 0.0
Epoch 1400 loss: 0.0
Epoch 1500 loss: 0.0
Epoch 1600 loss: 0.0
Epoch 1700 loss: 0.0
Epoch 1800 loss: 0.0
Epoch 1900 loss: 0.0


3.4.2 Предсказание цен алмазов

3.4.2.1 Создайте датасет на основе файла diamonds.csv. 

1. Удалите все нечисловые столбцы
2. Целевой столбец (`y`) - `price`
3. Преобразуйте данные в тензоры корректных размеров

3.4.2.2 Разбейте датасет на обучающий и тестовый датасет при помощи `torch.utils.data.random_split`.

3.4.2.3 Обучите модель для предсказания цен при помощи мини-пакетного градиентного спуска (`batch_size = 256`). 

3.4.2.4 Выведите график функции потерь в зависимости от номера эпохи (значение потерь для эпохи рассчитывайте как среднее значение ошибок на каждом батче). Проверьте качество модели на тестовой выборке. 


In [ ]:
class DiamondsDataset(Dataset):
  def __init__(self, data):
    pass

  def __len__(self):
    pass

  def __getitem__(self, idx):
    pass

3.4.3 Модифицируйте метод `__init__` датасета из 3.4.2 таким образом, чтобы он мог принимать параметр `transform: callable`. Реализуйте класс `DropColsTransform` для удаления нечисловых данных из массива. Реализуйте класс `ToTensorTransorm` для трансформации массива в тензор.

In [ ]:
class DiamondsDataset(Dataset):
  def __init__(self, data, transform):
    # ....
    self.transform = transform
    # ....

  def __len__(self):
    pass

  def __getitem__(self, idx):
    # ...
    sample = self.X[idx], self.y[idx]
    if self.transform:
      sample = self.transform(sample)
    # ....

In [ ]:
class DropColsTransform:
  def __init__(self, drop):
    pass
  
  def __call__(self, sample):
    X, y = sample
    # <удаление из X столбцов self.drop>
    return X, y

In [ ]:
class ToTensorTransform:
  def __call__(self, sample):
    X, y = sample
    # <преобразование X и y в тензоры>
    return X, y

In [ ]:
from torchvision import transforms

drop = DropColsTransform(drop=[1, 2, 3])
to_tensor = ToTensorTransform()
dataset = DiamondsDataset(data, transforms.compose([drop, to_tensor]))
